In [20]:
from binascii import hexlify
from binascii import unhexlify
from hashlib import sha256
from hashlib import sha512
import hmac
from ecc import PrivateKey, S256Point, Signature
from helper import encode_base58_checksum, decode_base58, decode_base58_extended, hash160
from io import BytesIO, StringIO

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [21]:
def extended_privkey(depth,finger_print,_index,chain_code,privkey, testnet=False):
    if testnet:
        version= 0x04358394.to_bytes(4,"big")
    else:
        version= 0x0488ade4.to_bytes(4,"big")
    depth=depth.to_bytes(1,"big")
    fingerprint =finger_print
    index = _index.to_bytes(4,"big")
    return version+depth+fingerprint+index+chain_code+b'\x00'+privkey

def deserialize_xprvk(s):
    version = s.read(4)
    depth = s.read(1)
    fingerprint = s.read(4)
    index= s.read(4)
    chain_code = s.read(32)
    s.read(1)
    privkey = s.read(32)
    return [version,depth, fingerprint,index,chain_code,privkey]


In [22]:
xtprvk = 'xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U'
decoded =deserialize_xprvk( BytesIO(decode_base58_extended(xtprvk)))


In [23]:
for x in decoded:
    print(hex(int.from_bytes(x,"big")))

0x488ade4
0x0
0x0
0x0
0x60499f801b896d83179a4374aeb7822aaeaceaa0db1f85ee3e904c4defbd9689
0x4b03d6fc340455b363f51020ad3ecca4f0850280cf436c70c727923f6db46c3e


In [24]:
master_priv_key = decoded[-1]
chain_code = decoded[-2]

In [25]:
pub_key = PrivateKey(int.from_bytes(master_priv_key,"big")).point.sec()
finger_print = hash160(pub_key)[:4]
print(hex(int.from_bytes(finger_print,"big")))

0xbd16bee5


In [26]:
i=0

In [27]:
if i >= 2**31: hardened=True
else: hardened=False
    
if hardened:
    I= hmac.new(
                key = chain_code,
                msg=b'\x00' + master_priv_key + i.to_bytes(4,"big") ,
                digestmod=sha512).digest()

    
else:
    
    I= hmac.new(
                key = chain_code,
                msg=pub_key + i.to_bytes(4,"big") ,
                digestmod=sha512).digest()
    
print(f"I: {I}")

I: b'`\xe3s\x9c\xc2\xc3\x95\x0b|M\x7f2\xccP>\x13\xb9\x96\xd0\xf7\xa4V#\xd0\xa9\x14\xe1\xef\xa7\xf8\x11\xe0\xf0\x90\x9a\xff\xaa~\xe7\xab\xe5\xddN\x10\x05\x98\xd4\xdcS\xcdp\x9dZ\\,\xac@\xe7A/#/|\x9c'


In [28]:
I_L, I_R = I[:32], I[32:]

In [29]:
child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(master_priv_key,"big"))%N
hex(child_privkey_int)

'0xabe74a98f6c7eabee0428f53798f0ab8aa1bd37873999041703c742f15ac7e1e'

In [30]:
child_privkey = PrivateKey(child_privkey_int).wif(compressed= True)

In [31]:
child_chain_code = I_R

In [32]:
child_privkey

'L2ysLrR6KMSAtx7uPqmYpoTeiRzydXBattRXjXz5GDFPrdfPzKbj'

In [33]:
hex(int.from_bytes(child_chain_code,"big"))

'0xf0909affaa7ee7abe5dd4e100598d4dc53cd709d5a5c2cac40e7412f232f7c9c'

In [34]:
hex(int.from_bytes(PrivateKey(child_privkey_int).point.sec(),"big"))

'0x2fc9e5af0ac8d9b3cecfe2a888e2117ba3d089d8585886c9c826b6b22a98d12ea'

In [35]:
xtended_private_key = extended_privkey(1,finger_print,0,child_chain_code,child_privkey_int.to_bytes(32,"big"))
xtended_private_key = encode_base58_checksum(xtended_private_key)
xtended_private_key

'xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt'

In [36]:
 b'\xbd\x16\xbe\xe5',

(b'\xbd\x16\xbe\xe5',)

In [37]:
class Xtended_privkey:
    
    def __init__(self, depth, fingerprint, index, chain_code, private_key, testnet = False):
        """
        All data must be bytes
        """
        self.testnet = testnet
        self.version = None
        self.depth = depth
        self.fingerprint = fingerprint
        self.index = index
        self.chain_code = chain_code
        self.private_key = private_key
        self.private_key_wif = PrivateKey(child_privkey_int).wif()
        self.xtended_key = self.get_xtended_key()
        self.xtended_public_key = self.extended_public_key()
        
        
    def __repr__(cls):
        return cls.get_xtended_key()
    
    
    def get_child_xtended_key(self, i):
       
        pub_key = PrivateKey(int.from_bytes(self.private_key,"big")).point.sec()
       
        child_fingerprint = hash160(pub_key)[:4]
     
        if i >= 2**31: 
            hardened=True
            msg=b'\x00' + self.private_key + i.to_bytes(4,"big")
        else: 
            hardened=False
            msg=pub_key + i.to_bytes(4,"big") 

        I= hmac.new(
                    key = self.chain_code,
                    msg=msg,
                    digestmod=sha512).digest()
        
        I_L, I_R = I[:32], I[32:]
        
        #Check if I_L is grater than N
        if int.from_bytes(I_L,"big")>=N : return self.get_child_xtended_key(self,i+1)
        
        
        child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(self.private_key,"big"))%N
        
        #check if public key is on the curve
        if PrivateKey(child_privkey_int).point.sec() is None:
            return self.get_child_xtended_key(self,i+1)
        #check if private key is 0
        if child_privkey_int==0 :
            return self.get_child_xtended_key(self,i+1)
                           
        child_chain_code = I_R
        
        child_depth = (int.from_bytes(self.depth,"big")+1).to_bytes(1,"big")
        
        self.extended_public_key()
        return Xtended_privkey(child_depth, child_fingerprint, i.to_bytes(4,"big"), 
                   child_chain_code, child_privkey_int.to_bytes(32,"big"),testnet = self.testnet)
        
        
        
    def get_child_from_path(self,path_string):
        child = self
        start = False
        begin = 0
        hardened = False
        for index,char in enumerate(path_string):
            
            if char == 'm' and index==0:
                continue
            elif char == '/':
                if start:
                    if hardened:
                        index = int(path_string[begin:index-1])+2**31
                        hardened = False
                    else:
                        index = int(path_string[begin:index])
                    child = child.get_child_xtended_key(index)
                    start = False
                continue
            elif char == "H" or char == "h":
                hardened = True
                continue
                
            elif not start:
                begin = index
                start = True
                
        if hardened:
            index = int(path_string[begin:-1])+2**31
            hardened = False
        else:
            index = int(path_string[begin:])
        child = child.get_child_xtended_key(int(index))
        return child
    
    
    def get_xtended_key(self):
        if self.testnet:
            version= 0x04358394.to_bytes(4,"big")
        else:
            version= 0x0488ade4.to_bytes(4,"big")
        
        pre_ser = version+self.depth+self.fingerprint+self.index+self.chain_code+b'\x00'+self.private_key
        return encode_base58_checksum(pre_ser)
    
    #@classmethod
    def get_xtended_public_key(self):
        return self.extended_public_key()
        
        
    def extended_public_key(self):
        pub_key =  PrivateKey(int.from_bytes(self.private_key,"big")).point.sec()
        x_pub_key = Xtended_pubkey(self.depth, self.fingerprint, self.index, 
                   self.chain_code, pub_key,testnet = self.testnet)
        self.xtended_public_key = x_pub_key
        return x_pub_key
        
    
    
    @classmethod
    def from_bip39_seed(cls, seed_int, testnet = False):
        I= hmac.new(
                    key = b"Bitcoin seed",
                    msg=seed_int.to_bytes(64,"big"),
                    digestmod=sha512).digest()
        return cls(depth=b'\x00', fingerprint=b'\x00'*4, index=b'\x00'*4, 
                   chain_code = I[32:], private_key=I[:32], testnet=testnet)
        
    
    @classmethod
    def parse(cls,xpk_string):
        testnet = None
        s = BytesIO(decode_base58_extended(xpk_string))
        version = s.read(4)
        if version == b"\x04\x35\x83\x94": 
            testnet = True
        elif version == b"\x04\x88\xad\xe4": 
                testnet = False
        depth = s.read(1)
        fingerprint = s.read(4)
        index= s.read(4)
        chain_code = s.read(32)
        s.read(1)
        privkey = s.read(32)
        return cls(depth=depth, fingerprint=fingerprint, index=index, 
                   chain_code = chain_code, private_key=privkey, testnet=testnet)


In [38]:
string = "xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi"
master = Xtended_privkey.parse(string)

NameError: name 'Xtended_pubkey' is not defined

In [ ]:
master

In [ ]:
master.xtended_public_key

In [ ]:
child = master.get_child_xtended_key(2**31)

In [ ]:
child

In [ ]:
test = master.get_child_from_path("m/0H/1")

In [ ]:
test.get_xtended_public_key()

In [ ]:
test.xtended_key

In [ ]:
#-- TO DO --:
#test the private key being 0 or grater than N


In [39]:
class Xtended_pubkey:
    
    def __init__(self, depth, fingerprint, index, chain_code, public_key, testnet = False):
        """
        All data must be bytes
        """
        self.testnet = testnet
        self.version = None
        self.depth = depth
        self.fingerprint = fingerprint
        self.index = index
        self.chain_code = chain_code
        self.public_key = public_key
        self.xtended_key = None
        
        
    def __repr__(cls):
        return cls.get_xtended_key()
    
    
    def get_child_xtended_key(self, i):
       
        child_fingerprint = hash160(self.public_key)[:4]
     
        if i >= 2**31: 
            hardened=True
            raise Exception ("Extended public keys not possible for hardened keys")
        else: 
            hardened=False
            msg=self.public_key + i.to_bytes(4,"big") 

        I= hmac.new(
                    key = self.chain_code,
                    msg=msg,
                    digestmod=sha512).digest()
        
        I_L, I_R = I[:32], I[32:]
        
        #Check if I_L is grater than N
        if int.from_bytes(I_L,"big")>=N : 
            return self.get_child_xtended_key(self,i+1)
        
        child_public_key = (PrivateKey(int.from_bytes(I_L,"big")).point + S256Point.parse(self.public_key)).sec()
        
        #check if public key is on the curve
        if child_public_key is None:
            return self.get_child_xtended_key(self,i+1)
                           
        child_chain_code = I_R
        
        child_depth = (int.from_bytes(self.depth,"big")+1).to_bytes(1,"big")
        
        return Xtended_pubkey(child_depth, child_fingerprint, i.to_bytes(4,"big"), 
                   child_chain_code, child_public_key,testnet = self.testnet)
        
        
        
    def get_child_from_path(self,path_string):
        child = self
        start = False
        begin = 0
        hardened = False
        for index,char in enumerate(path_string):
            
            if char == 'm' and index==0:
                continue
            elif char == '/':
                if start:
                    if hardened:
                        index = int(path_string[begin:index-1])+2**31
                        hardened = False
                    else:
                        index = int(path_string[begin:index])
                        
                    #######Child creation happens here #######
                    child = child.get_child_xtended_key(index)
                    start = False
                    
                continue
            elif char == "H" or char == "h":
                hardened = True
                continue
                
            elif not start:
                begin = index
                start = True
                
        #Last child created     
        if hardened:
            index = int(path_string[begin:-1])+2**31
            hardened = False
        else:
            index = int(path_string[begin:])
        child = child.get_child_xtended_key(int(index))
        return child
    
    
    def get_xtended_key(self):
        if self.testnet:
            version= 0x043587cf.to_bytes(4,"big")
        else:
            version= 0x0488b21e.to_bytes(4,"big")
        
        pre_ser = version+self.depth+self.fingerprint+self.index+self.chain_code+self.public_key
        return encode_base58_checksum(pre_ser)
        
    
    @classmethod
    def parse(cls,xpk_string):
        testnet = None
        s = BytesIO(decode_base58_extended(xpk_string))
        version = s.read(4)
        if version == b'\x04\x35\x87\xcf': 
            testnet = True
        elif version == b'\x04\x88\xb2\x1e': 
                testnet = False
        depth = s.read(1)
        fingerprint = s.read(4)
        index= s.read(4)
        chain_code = s.read(32)
        pubkey = s.read(33)
        return cls(depth=depth, fingerprint=fingerprint, index=index, 
                   chain_code = chain_code, public_key=pubkey, testnet=testnet)


In [40]:
xpb = "xpub661MyMwAqRbcFW31YEwpkMuc5THy2PSt5bDMsktWQcFF8syAmRUapSCGu8ED9W6oDMSgv6Zz8idoc4a6mr8BDzTJY47LJhkJ8UB7WEGuduB"

In [41]:
test1 = Xtended_pubkey.parse(xpb)

In [42]:
test1

xpub661MyMwAqRbcFW31YEwpkMuc5THy2PSt5bDMsktWQcFF8syAmRUapSCGu8ED9W6oDMSgv6Zz8idoc4a6mr8BDzTJY47LJhkJ8UB7WEGuduB

In [43]:
child = test1.get_child_from_path("m/0")

In [44]:
child

xpub69H7F5d8KSRgmmdJg2KhpAK8SR3DjMwAdkxj3ZuxV27CprR9LgpeyGmXUbC6wb7ERfvrnKZjXoUmmDznezpbZb7ap6r1D3tgFxHmwMkQTPH

In [45]:
#from https://en.bitcoin.it/wiki/BIP_0032

test_vector = {
    "xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi":
    [
        {"m/0H":
         [
             "xpub68Gmy5EdvgibQVfPdqkBBCHxA5htiqg55crXYuXoQRKfDBFA1WEjWgP6LHhwBZeNK1VTsfTFUHCdrfp1bgwQ9xv5ski8PX9rL2dZXvgGDnw",
             "xprv9uHRZZhk6KAJC1avXpDAp4MDc3sQKNxDiPvvkX8Br5ngLNv1TxvUxt4cV1rGL5hj6KCesnDYUhd7oWgT11eZG7XnxHrnYeSvkzY7d2bhkJ7"
         ]
        },
        {"m/0H/1":
         [
             "xpub6ASuArnXKPbfEwhqN6e3mwBcDTgzisQN1wXN9BJcM47sSikHjJf3UFHKkNAWbWMiGj7Wf5uMash7SyYq527Hqck2AxYysAA7xmALppuCkwQ",
             "xprv9wTYmMFdV23N2TdNG573QoEsfRrWKQgWeibmLntzniatZvR9BmLnvSxqu53Kw1UmYPxLgboyZQaXwTCg8MSY3H2EU4pWcQDnRnrVA1xe8fs"
         ]
        },
        {"m/0H/1/2H":
         [
             "xpub6D4BDPcP2GT577Vvch3R8wDkScZWzQzMMUm3PWbmWvVJrZwQY4VUNgqFJPMM3No2dFDFGTsxxpG5uJh7n7epu4trkrX7x7DogT5Uv6fcLW5",
             "xprv9z4pot5VBttmtdRTWfWQmoH1taj2axGVzFqSb8C9xaxKymcFzXBDptWmT7FwuEzG3ryjH4ktypQSAewRiNMjANTtpgP4mLTj34bhnZX7UiM"
         ]
        },
        {"m/0H/1/2H/2":
         [
             "xpub6FHa3pjLCk84BayeJxFW2SP4XRrFd1JYnxeLeU8EqN3vDfZmbqBqaGJAyiLjTAwm6ZLRQUMv1ZACTj37sR62cfN7fe5JnJ7dh8zL4fiyLHV",
             "xprvA2JDeKCSNNZky6uBCviVfJSKyQ1mDYahRjijr5idH2WwLsEd4Hsb2Tyh8RfQMuPh7f7RtyzTtdrbdqqsunu5Mm3wDvUAKRHSC34sJ7in334"
         ]
        },
        {"m/0H/1/2H/2/1000000000":
         [
             "xpub6H1LXWLaKsWFhvm6RVpEL9P4KfRZSW7abD2ttkWP3SSQvnyA8FSVqNTEcYFgJS2UaFcxupHiYkro49S8yGasTvXEYBVPamhGW6cFJodrTHy",
             "xprvA41z7zogVVwxVSgdKUHDy1SKmdb533PjDz7J6N6mV6uS3ze1ai8FHa8kmHScGpWmj4WggLyQjgPie1rFSruoUihUZREPSL39UNdE3BBDu76"
         ]
        }
    ],
    
    "xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U":
    [
        {"m/0":
         [
             "xpub69H7F5d8KSRgmmdJg2KhpAK8SR3DjMwAdkxj3ZuxV27CprR9LgpeyGmXUbC6wb7ERfvrnKZjXoUmmDznezpbZb7ap6r1D3tgFxHmwMkQTPH",
             "xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt"
         ]
        },
        {"m/0/2147483647H":
         [
             "xpub6ASAVgeehLbnwdqV6UKMHVzgqAG8Gr6riv3Fxxpj8ksbH9ebxaEyBLZ85ySDhKiLDBrQSARLq1uNRts8RuJiHjaDMBU4Zn9h8LZNnBC5y4a",
             "xprv9wSp6B7kry3Vj9m1zSnLvN3xH8RdsPP1Mh7fAaR7aRLcQMKTR2vidYEeEg2mUCTAwCd6vnxVrcjfy2kRgVsFawNzmjuHc2YmYRmagcEPdU9"
         ]
        },
        {"m/0/2147483647H/1":
         [
             "xpub6DF8uhdarytz3FWdA8TvFSvvAh8dP3283MY7p2V4SeE2wyWmG5mg5EwVvmdMVCQcoNJxGoWaU9DCWh89LojfZ537wTfunKau47EL2dhHKon",
             "xprv9zFnWC6h2cLgpmSA46vutJzBcfJ8yaJGg8cX1e5StJh45BBciYTRXSd25UEPVuesF9yog62tGAQtHjXajPPdbRCHuWS6T8XA2ECKADdw4Ef"
         ]
        },
        {"m/0/2147483647H/1/2147483646H":
         [
             "xpub6ERApfZwUNrhLCkDtcHTcxd75RbzS1ed54G1LkBUHQVHQKqhMkhgbmJbZRkrgZw4koxb5JaHWkY4ALHY2grBGRjaDMzQLcgJvLJuZZvRcEL",
             "xprvA1RpRA33e1JQ7ifknakTFpgNXPmW2YvmhqLQYMmrj4xJXXWYpDPS3xz7iAxn8L39njGVyuoseXzU6rcxFLJ8HFsTjSyQbLYnMpCqE2VbFWc"
         ]
        },
        {"m/0/2147483647H/1/2147483646H/2":
         [
             "xpub6FnCn6nSzZAw5Tw7cgR9bi15UV96gLZhjDstkXXxvCLsUXBGXPdSnLFbdpq8p9HmGsApME5hQTZ3emM2rnY5agb9rXpVGyy3bdW6EEgAtqt",
             "xprvA2nrNbFZABcdryreWet9Ea4LvTJcGsqrMzxHx98MMrotbir7yrKCEXw7nadnHM8Dq38EGfSh6dqA9QWTyefMLEcBYJUuekgW4BYPJcr9E7j"
         ]
        }
    ],
    
    "xprv9s21ZrQH143K25QhxbucbDDuQ4naNntJRi4KUfWT7xo4EKsHt2QJDu7KXp1A3u7Bi1j8ph3EGsZ9Xvz9dGuVrtHHs7pXeTzjuxBrCmmhgC6":
    [
        {"m/0H":
         [
             "xpub68NZiKmJWnxxS6aaHmn81bvJeTESw724CRDs6HbuccFQN9Ku14VQrADWgqbhhTHBaohPX4CjNLf9fq9MYo6oDaPPLPxSb7gwQN3ih19Zm4Y",
             "xprv9uPDJpEQgRQfDcW7BkF7eTya6RPxXeJCqCJGHuCJ4GiRVLzkTXBAJMu2qaMWPrS7AANYqdq6vcBcBUdJCVVFceUvJFjaPdGZ2y9WACViL4L"
         ]
        
        }
    ]
    
    
}

In [46]:
for key_string, derived in test_vector.items():
    
    master = Xtended_privkey.parse(key_string)
    print(f"TESING MASTER EXTENDED PRIVATE KEY \n{key_string}")
    for dictionary in derived:
        for path, xtended_keys in dictionary.items():
            child = master.get_child_from_path(path)
            assert str(child.xtended_public_key) == xtended_keys[0] , f"{child.xtended_public_key} NOT EQUAL {xtended_keys[0]}"
            assert str(child.xtended_key == xtended_keys[1]) , f"{child.xtended_key} NOT EQUAL {xtended_keys[1]}"
            print(f"xtended private and public key in vector for path {path}... OK")
            
print("\nTEST PASSED SUCCESSFULLY")  

TESING MASTER EXTENDED PRIVATE KEY 
xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi
xtended private and public key in vector for path m/0H... OK
xtended private and public key in vector for path m/0H/1... OK
xtended private and public key in vector for path m/0H/1/2H... OK
xtended private and public key in vector for path m/0H/1/2H/2... OK
xtended private and public key in vector for path m/0H/1/2H/2/1000000000... OK
TESING MASTER EXTENDED PRIVATE KEY 
xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U
xtended private and public key in vector for path m/0... OK
xtended private and public key in vector for path m/0/2147483647H... OK
xtended private and public key in vector for path m/0/2147483647H/1... OK
xtended private and public key in vector for path m/0/2147483647H/1/2147483646H... OK
xtended private and public key in vector for path m/0/2147483647H/1/2147483